In [6]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from os.path import join
from Utils import *

# load data
data_path = Path(r'..\Midterm Report\data')
xgb_results = pd.read_csv(join(data_path, 'output_predictions.csv'))
xgb_results['Date'] = pd.to_datetime(xgb_results['Date'])
xgb_results.head()

,Date,Ticker,Close,daily_return,14 period RSI,MACD,SIGNAL,14 period STOCH %K,MFV,14 period ATR,...,ema21,ema15,ema5,normVol,alpha,label,Val_Class,Test_Class,Val_Reg,Test_Reg
0,2020-09-30,RAFA,1.945000,-0.041872,43.479695,0.078535,0.202629,7.075476,-1.105459e+07,0.303571,...,1.013232,0.915709,0.870706,1.031694,0.023304,1,0.0,NaN,-0.004916,NaN
1,2020-09-30,EXAS,75.290001,0.029114,30.625051,-4.359268,-3.339625,22.189186,7.007589e+08,3.030001,...,0.882094,0.895883,0.956642,1.797729,0.016150,1,0.0,NaN,-0.001558,NaN
2,2020-09-30,GWPH,103.949997,0.009517,34.979778,-5.888861,-5.631870,17.010252,2.168549e+08,3.114999,...,0.893090,0.913685,0.975917,0.928833,-0.002995,0,0.0,NaN,-0.001558,NaN
3,2020-09-30,AIMT,34.220001,1.715873,89.415840,1.034355,-0.347780,99.732869,2.011977e+05,2.094643,...,2.243760,2.262465,2.085619,5.345179,0.000312,1,0.0,NaN,-0.006893,NaN
4,2020-09-30,COLL,19.059999,0.047253,58.332402,0.251403,0.270376,76.841134,2.412889e+07,0.822286,...,1.053066,1.051878,1.034912,1.107884,0.004724,1,0.0,NaN,-0.000899,NaN


In [15]:
val_mask = xgb_results['Date'] == pd.to_datetime(dt.date(2020, 9, 30))
test_mask = xgb_results['Date'] == pd.to_datetime(dt.date(2020, 10, 31))
tickers = xgb_results[mask]['Ticker'].unique()
tickers

array(['RAFA', 'EXAS', 'GWPH', 'AIMT', 'COLL', 'MNTA', 'CRNX', 'XENE',
       'AVRO', 'MGTA', 'SAGE', 'GBT', 'VCYT', 'XNCR', 'FOLD', 'BHVN',
       'MYOK', 'ARGX', 'SRPT', 'BPMC', 'GLPG', 'ASND', 'NBIX', 'MRTX',
       'RCKT', 'QURE', 'ZGNX', 'MYOV'], dtype=object)

In [11]:
# get relevant data
asset_data = get_data_for_multiple_stocks(tickers, dt.date(2019, 1, 1), dt.date(2020, 9, 30))
asset_returns = get_df_by_metric(asset_data, 'log_return')
asset_returns.head()

0/28
5/28
10/28
15/28
20/28
25/28


,RAFA,EXAS,GWPH,AIMT,COLL,MNTA,CRNX,XENE,AVRO,MGTA,...,SRPT,BPMC,GLPG,ASND,NBIX,MRTX,RCKT,QURE,ZGNX,MYOV
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,0.0,0.016347,0.036496,0.017406,0.036035,0.023278,-0.022935,0.056982,-0.061939,0.051293,...,-0.015049,-0.051967,-0.013498,0.034359,-0.018658,0.051234,0.018717,-0.010113,0.045310,0.031196
2019-01-03,0.0,-0.034256,-0.013154,-0.032995,-0.001687,-0.058304,-0.011668,-0.044384,0.022118,-0.040822,...,-0.007751,-0.049670,-0.009993,-0.042372,-0.011191,-0.022651,-0.003981,-0.040778,0.000000,-0.005924
2019-01-04,0.0,0.057526,0.088181,0.030115,0.052081,-0.017984,0.008935,0.072430,0.044017,0.063887,...,0.078924,0.093629,0.083770,0.038973,0.075025,0.030237,0.028189,0.027014,0.075212,0.013574
2019-01-07,0.0,0.137939,0.051725,0.036809,-0.217550,0.075403,-0.074924,0.091731,-0.057862,-0.074359,...,-0.048002,0.132222,0.032015,0.045971,0.091397,0.070577,0.030556,0.047537,0.049797,-0.004112
2019-01-08,0.0,0.010560,0.023701,0.022385,-0.017415,-0.008004,-0.029563,0.094948,-0.190722,0.008734,...,0.008327,0.023957,0.027643,0.047339,0.030794,0.052649,0.004379,-0.025048,0.026254,0.034706


In [12]:
# get index data
index_data = get_data_for_multiple_stocks(['^GSPC'], dt.date(2019, 1, 1), dt.date(2020, 9, 30))
index_returns = get_df_by_metric(index_data, 'log_return')
index_returns.head()

0/1


,^GSPC
Date,
2019-01-02,0.001268
2019-01-03,-0.025068
2019-01-04,0.033759
2019-01-07,0.006986
2019-01-08,0.009649


In [14]:
# %% rf rate 2019 and 2020

# 2019
rf_2019 = pd.read_csv(join(data_path, 'yield_rate_2019.csv'))
rf_2019.drop(rf_2019.columns.difference(['Date','1 Yr']), 1, inplace=True)

rf_2019['1 Yr'] = rf_2019['1 Yr'].apply(lambda x: x / 100)
rf_2019['Date'] = pd.to_datetime(rf_2019['Date'])
rf_2019.set_index("Date", inplace=True)
rf_2019.rename(columns={'1 Yr': 'risk_free_rate'}, inplace=True)

# 2020
rf_2020 = pd.read_csv(join(data_path, 'yield_rate_2020.csv'))
rf_2020.drop(rf_2020.columns.difference(['Date','1 Yr']), 1, inplace=True)

rf_2020['1 Yr'] = rf_2020['1 Yr'].apply(lambda x: x / 100)
rf_2020['Date'] = pd.to_datetime(rf_2020['Date'])
rf_2020.set_index("Date", inplace=True)
rf_2020.rename(columns={'1 Yr': 'risk_free_rate'}, inplace=True)

# combine series and align index
rf = rf_2019.append(rf_2020)
rf = index_returns.merge(rf, how='left', left_index=True, right_index=True)['risk_free_rate']
rf.fillna(method='ffill', inplace=True)

rf

Date
2019-01-02    0.0260
2019-01-03    0.0250
2019-01-04    0.0257
2019-01-07    0.0258
2019-01-08    0.0260
               ...  
2020-09-24    0.0012
2020-09-25    0.0012
2020-09-28    0.0012
2020-09-29    0.0012
2020-09-30    0.0012
Name: risk_free_rate, Length: 441, dtype: float64

In [23]:
# calculate 1 year beta

mask = (asset_returns.index >= pd.to_datetime(dt.date(2019, 9, 30))) & \
        (asset_returns.index <= pd.to_datetime(dt.date(2020, 9, 30)))
    
betas = pd.Series()
for ticker in asset_returns.columns:
    asset_series = asset_returns[ticker][mask]
    index_series = index_returns['^GSPC'][mask]
    asset_beta = get_beta(np.exp(index_series) - 1, 
                          np.exp(asset_series) - 1,
                                 rf[mask])
    betas.loc[ticker] = asset_beta

betas

C:\Users\Timothy Lim\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


RAFA    0.413782
EXAS    1.116415
GWPH    0.852120
AIMT    0.981007
COLL    1.186132
MNTA    1.357641
CRNX    1.032640
XENE    0.746540
AVRO    1.286947
MGTA    1.323841
SAGE    1.169884
GBT     1.231911
VCYT    1.280062
XNCR    1.183711
FOLD    1.336018
BHVN    1.356942
MYOK    0.956883
ARGX    0.767602
SRPT    0.897928
BPMC    1.018449
GLPG    0.854267
ASND    0.585720
NBIX    0.733817
MRTX    1.034391
RCKT    1.138390
QURE    0.799475
ZGNX    0.953046
MYOV    0.807122
dtype: float64

In [39]:
# get results from xgb
alphas = xgb_results[test_mask][['Ticker', 'Test_Reg']]
alphas.set_index('Ticker', inplace=True)
alphas.rename(columns={"Test_Reg": "alpha"}, inplace=True)
alphas.head()

,alpha
Ticker,
RCKT,-0.000333
VCYT,-0.001084
XNCR,-0.000657
QURE,-0.001558
MYOV,-0.001603


In [38]:
# merge series
alpha_beta = alphas.merge(betas.rename("beta"), how='right', left_index=True, right_index=True)
alpha_beta

,alpha,beta
RAFA,0.000396,0.413782
EXAS,-0.000068,1.116415
GWPH,-0.001084,0.852120
AIMT,NaN,0.981007
COLL,0.000672,1.186132
MNTA,NaN,1.357641
CRNX,-0.001558,1.032640
XENE,-0.001060,0.746540
AVRO,-0.001558,1.286947
MGTA,-0.001389,1.323841


In [50]:
# get avg S&P500 return over past 1 year
index_monthly = index_returns.groupby(pd.Grouper(freq="M")).sum()
mask = index_monthly.index >= pd.to_datetime(dt.date(2019, 9, 30))
avg_index_monthly = np.exp(index_monthly[mask].mean()) - 1  # convert to simple

print(avg_index_monthly)
index_monthly

^GSPC    0.010753
dtype: float64


,^GSPC
Date,
2019-01-31,0.075742
2019-02-28,0.029296
2019-03-31,0.017766
2019-04-30,0.038560
2019-05-31,-0.068041
2019-06-30,0.066658
2019-07-31,0.013043
2019-08-31,-0.018257
2019-09-30,0.017035


In [60]:
# get predicted return as input to BL model
rf_rate = 0.0008  # 1 month treasury rate as of 30/9/2020

pred_return = alpha_beta['alpha'] + alpha_beta['beta'] * (avg_index_monthly.values[0] - rf_rate)
pred_return += rf_rate

pred_return

RAFA    0.005315
EXAS    0.011844
GWPH    0.008197
AIMT         NaN
COLL    0.013277
MNTA         NaN
CRNX    0.009520
XENE    0.007170
AVRO    0.012051
MGTA    0.012587
SAGE    0.012338
GBT     0.013432
VCYT    0.012456
XNCR    0.011924
FOLD    0.014637
BHVN    0.014799
MYOK    0.010218
ARGX    0.008960
SRPT    0.009877
BPMC    0.010869
GLPG    0.009024
ASND    0.007150
NBIX    0.006546
MRTX    0.010990
RCKT    0.011797
QURE    0.007199
ZGNX    0.008655
MYOV    0.007230
dtype: float64

In [65]:
# calculate confidence 
y_val = xgb_results[val_mask][['label', 'Val_Class']]
confidence = np.where(y_val['label'] == y_val['Val_Class'], 1, 0)
confidence.mean()

0.6785714285714286